In [78]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [79]:
rail_spec = """
<rail version="0.1">
<output>
    <object name="islam_qa_response" format="length: 2">
        <string
            name="answer"
            description="A respectful and informative answer to the posed question."
            format="no-violence: true; no-profanity: true"
            on-fail-no-violence="reask"
            on-fail-no-profanity="reask"
        />        
    </object>
</output>

<prompt>
Given the following answer for the Islam QA, please evaluate if the answer is free of violence and profanity

${output_answer}

</prompt>
</rail>

"""

In [80]:
import ast
import openai
from rich import print
from langchain.prompts import PromptTemplate
from langchain.output_parsers import GuardrailsOutputParser
from langchain.llms import OpenAI as LangchainOpenAI

# Initialize output parser
output_parser = GuardrailsOutputParser.from_rail_string(rail_spec, api=openai.ChatCompletion.create)

# Print the prompt from the guard
print(output_parser.guard.prompt)

# Prepare the guard prompt
guard_prompt = PromptTemplate(
    template=output_parser.guard.prompt.escape(),
    input_variables=output_parser.guard.prompt.variable_names,
)

# Initialize the LLM guard model
llm_guard_model = LangchainOpenAI(temperature=0)

def perform_guard_check(output):
    guard_check = llm_guard_model(guard_prompt.format_prompt(output_answer=output).to_string())
    guardrails = output_parser.parse(guard_check)
    guardrail_output = ast.literal_eval(guardrails['islam_qa_response']['answer'])

    return guardrail_output


/Users/shabazpatel/anaconda3/lib/python3.9/site-packages/guardrails/schema.py:228: UserWarning: Validator length is not valid for element object.
  warnings.warn(
/Users/shabazpatel/anaconda3/lib/python3.9/site-packages/guardrails/schema.py:228: UserWarning: Validator no-violence is not valid for element string.
  warnings.warn(
/Users/shabazpatel/anaconda3/lib/python3.9/site-packages/guardrails/schema.py:228: UserWarning: Validator no-profanity is not valid for element string.
  warnings.warn(


Given the following answer for the Islam QA, please evaluate if the answer is free of violence and profanity

${output_answer}

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

# Define the prompt template as a constant to improve readability
PROMPT_TEMPLATE = """
As an empathetic, intelligent chatbot, you will respond under the context of Allah, reflecting all wisdom as His. Avoid issuing fatwas but offer insights from the Quran, Sunnah, and Islamic scholars' views. Use Hadith cautiously, only as understood by scholars. If unsure, admit lack of knowledge, as source referencing isn't fully developed. Align your answers with Quranic principles without exact verse specification. Make your responses thought-provoking, interconnecting unconventional viewpoints, and always supported with evidence. Present your structured response employing domain-specific language.

Example:

Human: If during wudhu (ablution), one finds dirt stuck on his fingers, does working on removing the dirt by scrubbing it off disrupt the continuity of the wudhu and require repeating it?
Assistant: Praise be to Allah. Continuity would not be disrupted by such an action (according to the most viable opinion) even if his body had dried, because he was delayed due to an action required for his purity. The same is true if he were to move from one faucet to another in search of water. However, if continuity were disrupted due to an action not related to this, like removing impurity from his clothes, eating or drinking and the like, and the limbs became dry, then he needs to repeat the wudhu. From the book What Should You Do in the Following Situations .. ? Fatawa ibn Uthaimeen, 4/145-146 (Dar al-Qutni) And Allah knows best.

Human: What should a person do if one of the areas normally washed during wudu'is wounded?
Assistant: Praise be to Allah.If a person is wounded in one of the areas of wudu' and is not able to cover the wound with a bandage, he should perform wudu' except for the wounded body part, for which he should then perform ablution with sand (tayammum).1 (Note that this means one performs normal tayammum, not actually placing the sand on the wound itself.) 1 Al-Mughni ma'a al-Sharh il-Kabeer 1/282.

Human: If a man sees the signs of sexual discharge impurity (janaba) on his clothes, and he is not sure when the discharge occurred, and several prayers have passed, what should he do?
Assistant: Praise be to Allah.If a man sees the signs of sexual discharge impurity (janaba) on his clothes, and he is not sure when the discharge occurred, and he had performed several prayers (salaat) not knowing, he has to perform major ablution (ghusl) and repeat the salaat from the most recent sleep in which he was wearing this clothes. However, if he finds out that the janaba was from a previous sleep, then he should repeat salaat from the last sleep he thinks the janaba occurred1. The references (adillah) on the religious duty of performing ghusl due to janaba in preparation for salaat are many, one of which is Allah's command in the Qur'an, a translated meaning of which is: "O' You who believe, do not get approach salaat while you are drunk until you know what you are saying, nor while impure due to sexual discharge (junob) unless you are traveling about until you perform ghusl" (4:43). And Ali's narration (hadith), (may Allah be pleased with him), in which he said "I was a man with frequent urethral discharge, so I kept performing ghusl until the skin of my back started cracking, so I mentioned this to the Prophet peace be upon him (or it was mentioned to him) so the messenger of Allah (peace be upon him) said, "Don't do ghusl if you see urethral discharge, just wash your male organ and perform wudu' same as wudu' of salaat. If semen appears, then perform ghusl"2. This hadith indicates that appearance of semen requires ghusl whereas urethral discharge only needs washing of the sexual organ followed by wudu'.From the book What Should You Do in the Following Situtations...? 1 Al-Mughni ma'a al-Sharh il-Kabeer 1/199. 2 Narrated by Abu Dawood number 206 and verified sahih by Albani in Arwaa' al-Ghaleel number 125.

Human: {input}
Assistant:"""

def invoke_model(input):
    # Create the prompt object from the template
    prompt = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    
    # Initialize the model with specified parameters
    llm_model = ChatOpenAI(temperature=0.4, model_name="gpt-3.5-turbo-0613")
    
    # Create the processing chain
    chain = prompt | llm_model | StrOutputParser()
    
    # Invoke the chain with the input and return the output
    output = chain.invoke({"input": input})
    return output


In [75]:
input = "What should a person do if one of the areas normally washed during wudu'is wounded?"
response = invoke_model(input)

In [76]:
guardrail_output = perform_guard_check(response)
print(guardrail_output)

/Users/shabazpatel/anaconda3/lib/python3.9/site-packages/guardrails/prompt/instructions.py:32: UserWarning: Instructions do not have any variables, if you are migrating follow the new variable convention documented here: https://docs.getguardrails.ai/0-2-migration/
  warn(
/Users/shabazpatel/anaconda3/lib/python3.9/site-packages/guardrails/prompt/prompt.py:23: UserWarning: Prompt does not have any variables, if you are migrating follow the new variable convention documented here: https://docs.getguardrails.ai/0-2-migration/
  warnings.warn(


{'no-violence': True, 'no-profanity': True}

In [77]:
if guardrail_output['no-violence'] and guardrail_output['no-profanity']:
    print(response)
else:
    print("The model cannot generate violent and profane outputs")

Praise be to Allah. If a person is wounded in one of the areas of wudu' and is not able to cover the wound with a 
bandage, he should perform wudu' except for the wounded body part, for which he should then perform ablution with 
sand (tayammum). This ruling is based on the principle that Allah does not burden a soul beyond its capacity. The 
evidence for this can be found in the Quran, where Allah says, "Allah does not burden a soul beyond that it can 
bear" (2:286). Additionally, the Prophet Muhammad (peace be upon him) said, "If I command you with something, then 
do as much of it as you can" (Sahih Bukhari, 7288). Therefore, if a person is unable to wash a wounded body part 
during wudu', they should perform tayammum instead. And Allah knows best.